<a href="https://colab.research.google.com/github/schauppi/Udemy_TensorFlow_Developer/blob/main/Abschnitt_8_NLP_fundamentals_in_TensorFlow/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (text or speech sequences).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a70c1e50-52e7-e43d-4178-b48fd14dd213)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-08-04 15:22:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-04 15:22:54 (98.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we´re going to be using is Kaggle´s introduction to BLP dataset (text samples of Tweets labelled as disaster or nor disaster)

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2021-08-04 15:22:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-08-04 15:22:57 (151 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset 

To visualize our text samples, we first have to read them in, one way to do so would be to use Python.

Another way to do this is to use pandas...

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
#What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let´s visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
1943: Poland - work party prisoners in the Nazi death camp Treblinka rebelled seizing small arms and setting buildings on fire. #history

---

Target: 0 (not real disaster)
Text:
And when those waves shall ripple collide it's on the tide of YOUR LOVE I will survive. #love @LesleyChappelle

---

Target: 0 (not real disaster)
Text:
Do you want to play a game?
http://t.co/sQFp6Ecz0i
Its a GoogleMaps mashup that calculates the effects of the detonation of nuclear bomb

---

Target: 1 (real disaster)
Text:
Fresh encounter in Pulwama of J&amp;amp;K one terrorist killed others holed up inside a house http://t.co/oEf123l5Rc

---

Target: 1 (real disaster)
Text:
Slip Sliding Away - Flash Floods Info for Writers w/Tony Nester @SonoranRattler #writingtips http://t.co/sLTtOrRLHs

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

#Use train_test_split to split train data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [13]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convert text into numbers

When dealing with a text problem, one of the first things you´ll have to do before you can build a model is to convert your text to numbers.

There are a few ways you can to this, namely:
* Tokenization - direct mapping of token (a token could be a word or character to number)
* Embedding - create a matrix of a feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

## Text vectorization (tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how man word in the vocabulary (automatically adds <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, #create groups of n-words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long the sequences should be
                                    pad_to_max_tokens=True)

In [18]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [19]:
len(train_sentences[0].split())

7

In [20]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
#Setup vectorizeation variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 #max length our sequences will be (e.g. how many words from a tweet will our model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [22]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [24]:
#Create a sample sentence and tokenize it
sample_sentence = "There´s a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [27]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Sweetpea's are running riot at the allotment - and brightening up a rainy day http://t.co/6NdBFOPK5m       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7869,   22,  876,  416,   17,    2,    1,    7,    1,   27,    3,
        9375,  101,    1,    0]])>

In [32]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] #get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(len(words_in_vocab)) 
print(top_5_words)
print(bottom_5_words)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we´re going to use TensorFlow´s embedding layer.

The parameters we care most about for our embedding layer:
* input_dim = the size of the vocabulary
* output_dim = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [33]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [34]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \
      \n\nEmbedded version")
#Embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 California &amp;#039;monster&amp;#039; fire is 20% contained as 13000 are told to evacuate http://t.co/aPTAP6Yx1r       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 3.7782636e-02,  9.2064627e-03,  1.1315454e-02, ...,
          2.4170410e-02,  2.2117827e-02,  2.8655019e-02],
        [ 2.6940715e-02,  3.1543229e-02, -1.6819704e-02, ...,
          1.9616950e-02,  1.6377542e-02,  2.8404389e-02],
        [-1.5355647e-05, -9.6148737e-03,  1.4928568e-02, ...,
          3.9443862e-02, -4.9122691e-02, -4.2237736e-02],
        ...,
        [ 2.6940715e-02,  3.1543229e-02, -1.6819704e-02, ...,
          1.9616950e-02,  1.6377542e-02,  2.8404389e-02],
        [-2.2359157e-02,  3.6393117e-02, -1.8463135e-02, ...,
         -2.8931512e-02, -3.4240268e-02, -4.0329229e-02],
        [-2.2359157e-02,  3.6393117e-02, -1.8463135e-02, ...,
         -2.8931512e-02, -3.4240268e-02, -4.0329229e-02]]], dtype=float32)>

In [35]:
# Check out a single token´s embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03778264,  0.00920646,  0.01131545,  0.00518076, -0.0133593 ,
         0.03929036, -0.01414294, -0.03161459, -0.00495877,  0.03096329,
        -0.01196728,  0.02564961,  0.01724274,  0.00100365,  0.01032866,
        -0.04776615, -0.01546626,  0.00038428,  0.00625266,  0.04527661,
         0.0365673 , -0.03441306, -0.04719067, -0.04472034, -0.02945058,
         0.0324062 ,  0.01851105,  0.00763782,  0.01915877,  0.02380815,
        -0.02315556,  0.04150555, -0.02667974,  0.03859875, -0.01299065,
         0.03733796,  0.00286794,  0.00148229,  0.00902345, -0.0189046 ,
        -0.00932144, -0.02279499,  0.03133171, -0.03202881, -0.0167595 ,
         0.0258291 , -0.02706746, -0.01693754, -0.02033973,  0.03912194,
         0.04909182,  0.01455671, -0.03049357, -0.00687478, -0.00479299,
         0.04180915,  0.0151122 , -0.02540559, -0.03763508, -0.03264932,
        -0.04811699,  0.02164446, -0.02564906, -0.03686898,  0.00324183,
  